# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
# city_data_df.count()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san luis de la loma,17.2706,-100.8939,26.24,87,100,2.90,MX,1696213072
1,1,igarka,67.4667,86.5833,-4.13,77,27,3.43,RU,1696213072
2,2,dzagam,40.9033,45.8856,16.85,83,100,0.61,AZ,1696213074
3,3,pontianak,-0.0333,109.3333,26.99,78,20,2.06,ID,1696213074
4,4,jinghong,21.9910,100.7341,25.46,81,57,0.96,CN,1696213015


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
map_cities = city_data_df.hvplot.points("Lat", 
                                        "Lng", 
                                        geo = True,
                                        color = "City", 
                                        tiles = "OSM",
                                        size = "Humidity",
                                        alpha=0.5)

# Display the map
map_cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
    #My ideal weather condition is 
    #   -temperatures between 20 to 25 fahrenheit.
    #   -temperatures cloudiness up to 2.
    #   -temperatures wind speed up to 5.
    #   -temperatures humidity up to 70.

city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27) 
                                    & (city_data_df["Max Temp"] >= 20)
                                    & (city_data_df["Cloudiness"] <= 2) 
                                    & (city_data_df["Wind Speed"] <= 5) 
                                    & (city_data_df["Humidity"] <= 80)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
# print(city_data_df.count())
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,hamilton,39.1834,-84.5333,21.89,79,0,0.45,US,1696213076
30,30,kill devil hills,36.0307,-75.6760,21.66,78,0,3.09,US,1696213083
31,31,kharan,28.5833,65.4167,20.75,16,0,1.95,PK,1696213083
81,81,protaras,35.0125,34.0583,21.11,79,0,3.98,CY,1696212904
117,117,warrnambool,-38.3833,142.4833,20.57,47,0,4.35,AU,1696213114
146,146,port lincoln,-34.7333,135.8667,22.85,45,0,4.75,AU,1696213124
244,244,tolanaro,-25.0319,46.9987,20.13,80,0,3.32,MG,1696213159
250,250,saint-philippe,-21.3585,55.7679,21.77,75,0,2.58,RE,1696213161
254,254,afif,23.9065,42.9172,24.05,19,0,4.48,SA,1696213162
301,301,rice lake,45.5061,-91.7382,24.18,78,0,3.60,US,1696213182


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.count()
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,hamilton,US,39.1834,-84.5333,79,
30,kill devil hills,US,36.0307,-75.6760,78,
31,kharan,PK,28.5833,65.4167,16,
81,protaras,CY,35.0125,34.0583,79,
117,warrnambool,AU,-38.3833,142.4833,47,
146,port lincoln,AU,-34.7333,135.8667,45,
244,tolanaro,MG,-25.0319,46.9987,80,
250,saint-philippe,RE,-21.3585,55.7679,75,
254,afif,SA,23.9065,42.9172,19,
301,rice lake,US,45.5061,-91.7382,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel","apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.count()
hotel_df

Starting hotel search
hamilton - nearest hotel: No hotel found
kill devil hills - nearest hotel: No hotel found
kharan - nearest hotel: Saija
protaras - nearest hotel: No hotel found
warrnambool - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
afif - nearest hotel: Spark hotel
rice lake - nearest hotel: No hotel found
jonesboro - nearest hotel: No hotel found
evansville - nearest hotel: No hotel found
urangan - nearest hotel: No hotel found
natal - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
maintirano - nearest hotel: No hotel found
multan - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: No hotel found
tamandare - nearest hotel: No hotel found
whangarei - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
taourirt - nearest hotel: No hotel found
turabah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
11,hamilton,US,39.1834,-84.5333,79,No hotel found
30,kill devil hills,US,36.0307,-75.6760,78,No hotel found
31,kharan,PK,28.5833,65.4167,16,Saija
81,protaras,CY,35.0125,34.0583,79,No hotel found
117,warrnambool,AU,-38.3833,142.4833,47,No hotel found
146,port lincoln,AU,-34.7333,135.8667,45,No hotel found
244,tolanaro,MG,-25.0319,46.9987,80,No hotel found
250,saint-philippe,RE,-21.3585,55.7679,75,No hotel found
254,afif,SA,23.9065,42.9172,19,Spark hotel
301,rice lake,US,45.5061,-91.7382,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points("Lat", 
                                    "Lng", 
                                    geo = True,
                                    tiles = "OSM",
                                    color = "City", 
                                    alpha=0.5, 
                                    size = "Humidity",
                                    hover_cols=["Hotel Name", "Country"])

# Display the map
map_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Humidity,Hotel Name,Country)